# Metrics and Evaluation

## Evaluating the original model

**Metrics reminder**

- Precision: the proportion of positive predictions that are actually positive. High precision means that the model has few false positives.
- Recall: the proportion of true positives that were correctly identified as such. High recall means the model has few false negatives.
- F1 score: the harmonic mean of precision and recall. This is a composite metric that takes into account both false positives and false negatives.
- Support: the number of samples in each class.


In [ ]:
# Internal imports
import os

# External imports
import pandas as pd
import plotly.graph_objects as go
import numpy as np
from imageai.Classification.Custom import CustomImageClassification
from sklearn.metrics import classification_report


# Path to the brain tumor dataset composed of two sub folders training and testing
ARCHIVE_DIR = "../data/archive"
DATA_DIR = "../data/brain_dataset"
COMBINED_DIR = "../data/combined"

# Path to the brain tumor dataset for testing
ARCHIVE_TRAINING_DATASET = "../data/archive/training"
TRAINING_DATASET = "../data/brain_dataset/training"
COMBINED_TRAINING_DATASET = "../data/combined/training"

# Path to the brain tumor dataset for testing
ARCHIVE_TESTING_DATASET = "../data/archive/testing"
TESTING_DATASET = "../data/brain_dataset/testing"
COMBINED_TESTING_DATASET = "../data/combined/testing"

In [25]:
def confusion_matrix_count(df):
    df_cm = pd.crosstab(
        index=df["Predicted Label"],
        columns=df["True Label"],
    )
    return df_cm


def confusion_matrix_distrib(df):
    df_cm = df / np.sum(df, axis=0)
    return df_cm


def confusion_matrix_count_plot(df, dataset_name, color="Greens"):
    heatmap = go.Heatmap(
        z=df.values,
        x=df.columns,
        y=df.index,
        colorscale=color,
    )
    fig = go.Figure(data=heatmap)
    fig.data[0].texttemplate = "%{z}"
    fig.data[0].textfont.size = 14
    fig.update_layout(
        width=800,
        height=600,
        title={
            "text": f"Confusion matrix, original model, {dataset_name} | Count",
            "x": 0.5,
            "xanchor": "center",
        },
        xaxis_title="Predicted Label",
        yaxis_title="True Label",
    )
    fig.show()


def confusion_matrix_distrib_plot(df, dataset_name, color="Greens"):
    heatmap = go.Heatmap(
        z=df.values,
        x=df.columns,
        y=df.index,
        colorscale=color,
        zmin=0,
        zmax=1,
    )
    fig = go.Figure(data=heatmap)
    fig.data[0].texttemplate = "%{z:.2%}"
    fig.data[0].textfont.size = 14
    fig.update_layout(
        width=800,
        height=600,
        title={
            "text": f"Confusion matrix, original model, {dataset_name} | Distribution",
            "x": 0.5,
            "xanchor": "center",
        },
        xaxis_title="Predicted Label",
        yaxis_title="True Label",
    )
    fig.show()

### Original model with archive dataset

In [ ]:
# Loading imageai class
prediction = CustomImageClassification()
prediction.setModelTypeAsDenseNet121()
prediction.setModelPath(os.getcwd() + "/archive/model.pt")
prediction.setJsonPath("classes.json")
prediction.loadModel()

prediction_label = []
prediction_probability = []
true_label = []

for i in ["glioma_tumor", "meningioma_tumor", "no_tumor", "pituitary_tumor"]:
    for j in os.listdir(ARCHIVE_TESTING_DATASET + "/" + i):
        pred = prediction.classifyImage(
            ARCHIVE_TESTING_DATASET + "/" + i + "/" + j, result_count=1
        )
        true_label.append(i)
        prediction_label.append(pred[0][0])
        prediction_probability.append(pred[1][0])

df_prediction_archive = pd.DataFrame(
    {
        "Predicted Label": prediction_label,
        "True Label": true_label,
        "Probability": prediction_probability,
    }
)

In [4]:
# Confusion Matrix
df_confusion_matrix_archive = confusion_matrix_count(df_prediction_archive)

# Confusion matrix, distribution (%)

confusion_matrix_distribution_archive = confusion_matrix_distrib(
    df_confusion_matrix_archive
)

display(df_confusion_matrix_archive)
display(confusion_matrix_distribution_archive.style.format("{:.2f}"))

True Label,glioma_tumor,meningioma_tumor,no_tumor,pituitary_tumor
Predicted Label,,,,
glioma_tumor,40,3,2,0
meningioma_tumor,22,109,1,4
no_tumor,37,3,102,16
pituitary_tumor,1,0,0,54


True Label,glioma_tumor,meningioma_tumor,no_tumor,pituitary_tumor
Predicted Label,,,,
glioma_tumor,0.40,0.03,0.02,0.00
meningioma_tumor,0.22,0.95,0.01,0.05
no_tumor,0.37,0.03,0.97,0.22
pituitary_tumor,0.01,0.00,0.00,0.73


In [5]:
confusion_matrix_count_plot(
    df=df_confusion_matrix_archive, dataset_name="original dataset", color="Reds"
)

In [6]:
confusion_matrix_distrib_plot(
    df=confusion_matrix_distribution_archive,
    dataset_name="original dataset",
    color="Reds",
)

In [7]:
# Classification report of the original model with a new dataset
metrics_archive = classification_report(
    df_prediction_archive["True Label"],
    df_prediction_archive["Predicted Label"],
    output_dict=True,
)
df_metrics_archive = pd.DataFrame(metrics_archive).transpose().round(decimals=2)

display(df_metrics_archive)

,precision,recall,f1-score,support
glioma_tumor,0.89,0.40,0.55,100.00
meningioma_tumor,0.80,0.95,0.87,115.00
no_tumor,0.65,0.97,0.78,105.00
pituitary_tumor,0.98,0.73,0.84,74.00
accuracy,0.77,0.77,0.77,0.77
macro avg,0.83,0.76,0.76,394.00
weighted avg,0.82,0.77,0.76,394.00


### Original model with the new dataset

The new dataset combines these two datasets :
- https://www.kaggle.com/datasets/sartajbhuvaji/brain-tumor-classification-mri
- https://www.kaggle.com/datasets/thomasdubail/brain-tumors-256x256

To combine the two datasets, I removed the existing training/testing folders and regrouped them into the four subfolders :
- no_tumor
- glioma_tumor
- meningioma_tumor
- pituitary_tumor

After that, I used my script data/find_duplicates.py to find and remove the duplicates by checking the image hashes.
Then, I used my script data/rename_files.py to rename the files following the same rule.
Finally, I used a my script data/split_dataset.py to split the data into training/testing with a 0.80/0.20 rule.

Now, we will test the original model with the new combined and cleansed dataset.

In [ ]:
# Loading imageai class
prediction = CustomImageClassification()
prediction.setModelTypeAsDenseNet121()
prediction.setModelPath(os.getcwd() + "/archive/model.pt")
prediction.setJsonPath("classes.json")
prediction.loadModel()

prediction_label = []
prediction_probability = []
true_label = []

for i in ["glioma_tumor", "meningioma_tumor", "no_tumor", "pituitary_tumor"]:
    for j in os.listdir(TESTING_DATASET + "/" + i):
        pred = prediction.classifyImage(
            TESTING_DATASET + "/" + i + "/" + j, result_count=1
        )
        true_label.append(i)
        prediction_label.append(pred[0][0])
        prediction_probability.append(pred[1][0])

df_prediction = pd.DataFrame(
    {
        "Predicted Label": prediction_label,
        "True Label": true_label,
        "Probability": prediction_probability,
    }
)

In [9]:
# Confusion Matrix
df_confusion_matrix = confusion_matrix_count(df_prediction)

# Confusion matrix, distribution (%)

confusion_matrix_distribution = confusion_matrix_distrib(df_confusion_matrix)

display(df_confusion_matrix)
display(confusion_matrix_distribution.style.format("{:.2f}"))

True Label,glioma_tumor,meningioma_tumor,no_tumor,pituitary_tumor
Predicted Label,,,,
glioma_tumor,337,28,1,25
meningioma_tumor,8,293,3,4
no_tumor,11,24,123,74
pituitary_tumor,1,3,0,233


True Label,glioma_tumor,meningioma_tumor,no_tumor,pituitary_tumor
Predicted Label,,,,
glioma_tumor,0.94,0.08,0.01,0.07
meningioma_tumor,0.02,0.84,0.02,0.01
no_tumor,0.03,0.07,0.97,0.22
pituitary_tumor,0.00,0.01,0.00,0.69


In [10]:
confusion_matrix_count_plot(
    df=df_confusion_matrix, dataset_name="new dataset", color="Oranges"
)

In [11]:
confusion_matrix_distrib_plot(
    df=confusion_matrix_distribution, dataset_name="new dataset", color="Oranges"
)

In [12]:
# Classification report of the original model with a new dataset
metrics = classification_report(
    df_prediction["True Label"], df_prediction["Predicted Label"], output_dict=True
)
df_metrics = pd.DataFrame(metrics).transpose().round(decimals=2)

display(df_metrics)

,precision,recall,f1-score,support
glioma_tumor,0.86,0.94,0.90,357.00
meningioma_tumor,0.95,0.84,0.89,348.00
no_tumor,0.53,0.97,0.69,127.00
pituitary_tumor,0.98,0.69,0.81,336.00
accuracy,0.84,0.84,0.84,0.84
macro avg,0.83,0.86,0.82,1168.00
weighted avg,0.89,0.84,0.85,1168.00


## Metrics comparison between the original and the new dataset

### Classification reports

In [13]:
confusion_matrix_count_plot(
    df=df_confusion_matrix_archive, dataset_name="original dataset", color="Reds"
)

confusion_matrix_count_plot(
    df=df_confusion_matrix, dataset_name="new dataset", color="Oranges"
)

In [14]:
print("Original model, original dataset")
display(df_metrics_archive)
print("Original model, new dataset")
display(df_metrics)

Original model, original dataset


,precision,recall,f1-score,support
glioma_tumor,0.89,0.40,0.55,100.00
meningioma_tumor,0.80,0.95,0.87,115.00
no_tumor,0.65,0.97,0.78,105.00
pituitary_tumor,0.98,0.73,0.84,74.00
accuracy,0.77,0.77,0.77,0.77
macro avg,0.83,0.76,0.76,394.00
weighted avg,0.82,0.77,0.76,394.00


Original model, new dataset


,precision,recall,f1-score,support
glioma_tumor,0.86,0.94,0.90,357.00
meningioma_tumor,0.95,0.84,0.89,348.00
no_tumor,0.53,0.97,0.69,127.00
pituitary_tumor,0.98,0.69,0.81,336.00
accuracy,0.84,0.84,0.84,0.84
macro avg,0.83,0.86,0.82,1168.00
weighted avg,0.89,0.84,0.85,1168.00


## Original model with a combination of datasets

The new dataset combines these three datasets :
- https://www.kaggle.com/datasets/sartajbhuvaji/brain-tumor-classification-mri
- https://www.kaggle.com/datasets/thomasdubail/brain-tumors-256x256
- https://www.kaggle.com/datasets/masoudnickparvar/brain-tumor-mri-dataset?rvi=1

To combine the three datasets, I removed the existing training/testing folders and regrouped them into the four subfolders :
- no_tumor
- glioma_tumor
- meningioma_tumor
- pituitary_tumor

After that, I used find_duplicates.py to find and remove the duplicates by checking the image hashes.
Then, I used rename_files.py to rename the files following the same rule.
Finally, I used split_dataset.py to split the data into training/testing with a 0.80/0.20 rule.

Now, we will test the original model with the new combined and cleansed dataset.

In [ ]:
# Loading imageai class
prediction = CustomImageClassification()
prediction.setModelTypeAsDenseNet121()
prediction.setModelPath(os.getcwd() + "/archive/model.pt")
prediction.setJsonPath("classes.json")
prediction.loadModel()

prediction_label = []
prediction_probability = []
true_label = []

for i in ["glioma_tumor", "meningioma_tumor", "no_tumor", "pituitary_tumor"]:
    for j in os.listdir(COMBINED_TESTING_DATASET + "/" + i):
        pred = prediction.classifyImage(
            COMBINED_TESTING_DATASET + "/" + i + "/" + j, result_count=1
        )
        true_label.append(i)
        prediction_label.append(pred[0][0])
        prediction_probability.append(pred[1][0])

df_prediction = pd.DataFrame(
    {
        "Predicted Label": prediction_label,
        "True Label": true_label,
        "Probability": prediction_probability,
    }
)

In [16]:
# Confusion Matrix
df_confusion_matrix = confusion_matrix_count(df_prediction)

# Confusion matrix, distribution (%)

confusion_matrix_distribution = confusion_matrix_distrib(df_confusion_matrix)

display(df_confusion_matrix)
display(confusion_matrix_distribution.style.format("{:.2f}"))

True Label,glioma_tumor,meningioma_tumor,no_tumor,pituitary_tumor
Predicted Label,,,,
glioma_tumor,606,36,0,30
meningioma_tumor,17,426,20,6
no_tumor,18,25,388,67
pituitary_tumor,1,3,0,419


True Label,glioma_tumor,meningioma_tumor,no_tumor,pituitary_tumor
Predicted Label,,,,
glioma_tumor,0.94,0.07,0.00,0.06
meningioma_tumor,0.03,0.87,0.05,0.01
no_tumor,0.03,0.05,0.95,0.13
pituitary_tumor,0.00,0.01,0.00,0.80


In [17]:
# Classification report of the original model with a new dataset
metrics = classification_report(
    df_prediction["True Label"], df_prediction["Predicted Label"], output_dict=True
)
df_metrics = pd.DataFrame(metrics).transpose().round(decimals=2)

display(df_metrics)

,precision,recall,f1-score,support
glioma_tumor,0.90,0.94,0.92,642.00
meningioma_tumor,0.91,0.87,0.89,490.00
no_tumor,0.78,0.95,0.86,408.00
pituitary_tumor,0.99,0.80,0.89,522.00
accuracy,0.89,0.89,0.89,0.89
macro avg,0.89,0.89,0.89,2062.00
weighted avg,0.90,0.89,0.89,2062.00


In [18]:
confusion_matrix_count_plot(
    df=df_confusion_matrix, dataset_name="new dataset", color="Greens"
)

confusion_matrix_distrib_plot(
    df=confusion_matrix_distribution, dataset_name="combined dataset", color="greens"
)

## Metrics and evaluation for ResNet50V2 model

### Imports, variables, model

In [80]:
import numpy as np
import pandas as pd
from keras import utils
from keras.preprocessing import image
from keras.models import load_model
from sklearn.metrics import confusion_matrix, classification_report

class_names = {
    0: "glioma_tumor",
    1: "meningioma_tumor",
    2: "no_tumor",
    3: "pituitary_tumor",
}

label_names = ["glioma_tumor", "meningioma_tumor", "no_tumor", "pituitary_tumor"]

model = load_model("ResNet50V2-32.keras")

### Functions

In [64]:
def new_prediction(path):
    # Charger l'image
    test_image = utils.load_img(path, target_size=(224, 224))

    # Convertir l'image en tableau et redimensionner les pixels
    test_image = utils.img_to_array(test_image)
    test_image /= 255.0

    # Ajouter une dimension de lot à l'image
    test_image = np.expand_dims(test_image, axis=0)

    # Effectuer la prédiction en utilisant la fonction predict du modèle
    result = model.predict(test_image)

    # Déterminer la classe prédite et la probabilité maximale
    class_pred = ""
    if np.argmax(result) == 0:
        class_pred = "glioma_tumor"
    elif np.argmax(result) == 1:
        class_pred = "meningioma_tumor"
    elif np.argmax(result) == 2:
        class_pred = "no_tumor"
    elif np.argmax(result) == 3:
        class_pred = "pituitary_tumor"
    return [class_pred, np.max(result)]


def confusion_matrix_count_for_keras(df, class_names, color="Blues"):
    df_cm = pd.DataFrame(df, columns=class_names.values(), index=class_names.values())
    heatmap = go.Heatmap(
        z=df_cm.values,
        x=df_cm.columns,
        y=df_cm.index,
        colorscale=color,
    )
    fig = go.Figure(data=heatmap)
    fig.data[0].texttemplate = "%{z}"
    fig.data[0].textfont.size = 14
    fig.update_layout(
        width=800,
        height=600,
        title={
            "text": "Matrice de confusion pour le count",
            "x": 0.5,
            "xanchor": "center",
        },
        xaxis_title="Predicted Label",
        yaxis_title="True Label",
    )
    fig.show()


def confusion_matrix_dist_for_keras(df, class_names, color="Blues"):
    df_cm = pd.DataFrame(df, columns=class_names.values(), index=class_names.values())
    df_cm_norm = df_cm.divide(df_cm.sum(axis=1), axis=0)
    heatmap = go.Heatmap(
        z=df_cm_norm.values,
        x=df_cm_norm.columns,
        y=df_cm_norm.index,
        colorscale=color,
    )
    fig = go.Figure(data=heatmap)
    fig.data[0].texttemplate = "%{z:.2%}"
    fig.data[0].textfont.size = 14
    fig.update_layout(
        width=800,
        height=600,
        title={
            "text": "Matrice de confusion pour la distribution",
            "x": 0.5,
            "xanchor": "center",
        },
        xaxis_title="Predicted Label",
        yaxis_title="True Label",
    )
    fig.show()

In [65]:
new_prediction("../data/combined/testing/glioma_tumor/G_0057.jpg")

1/1 [==============================] - 0s 394ms/step


['glioma_tumor', 0.9999993]

In [66]:
# Charger l'ensemble de test
test_datagen = image.ImageDataGenerator(rescale=1.0 / 255)
test_generator = test_datagen.flow_from_directory(
    COMBINED_TESTING_DATASET,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical",
    shuffle=False,
)

# Prédire les étiquettes de l'ensemble de test
test_pred = model.predict(test_generator)
test_predicted_labels = np.argmax(test_pred, axis=1)

# Obtenir les vraies étiquettes de l'ensemble de test
test_true_labels = test_generator.classes

Found 2062 images belonging to 4 classes.
65/65 [==============================] - 53s 816ms/step


In [73]:
# Créer des DataFrames pour les vraies et les prédictions d'étiquettes
test_true_labels_df = pd.DataFrame(test_true_labels, columns=["TrueLabels"])
test_predicted_labels_df = pd.DataFrame(
    test_predicted_labels, columns=["PredictedLabels"]
)

# Concaténer les deux DataFrames
labels_df = pd.concat([test_true_labels_df, test_predicted_labels_df], axis=1)

# Générer la matrice de confusion
confusion_mtx = confusion_matrix(labels_df["TrueLabels"], labels_df["PredictedLabels"])
print("Matrice de confusion :\n", confusion_mtx)

# Générer le rapport de classification
classification_rpt = classification_report(
    labels_df["TrueLabels"], labels_df["PredictedLabels"]
)
print("Rapport de classification :\n", classification_rpt)

Matrice de confusion :
 [[581  52   3   6]
 [ 28 440   2  20]
 [  6   7 393   2]
 [  2  11   0 509]]
Rapport de classification :
               precision    recall  f1-score   support

           0       0.94      0.90      0.92       642
           1       0.86      0.90      0.88       490
           2       0.99      0.96      0.98       408
           3       0.95      0.98      0.96       522

    accuracy                           0.93      2062
   macro avg       0.93      0.94      0.93      2062
weighted avg       0.93      0.93      0.93      2062



In [67]:
confusion_matrix_count_for_keras(confusion_mtx, class_names)

In [68]:
confusion_matrix_dist_for_keras(confusion_mtx, class_names)

In [81]:
# Générer le rapport de classification
classification_rpt = classification_report(
    labels_df["TrueLabels"],
    labels_df["PredictedLabels"],
    target_names=label_names,
    output_dict=True,
)

# Convertir le rapport de classification en DataFrame Pandas
df_classification_rpt = pd.DataFrame(classification_rpt).transpose()

# Convertir les valeurs en pourcentages et arrondir à deux décimales
df_classification_rpt = df_classification_rpt * 100
df_classification_rpt = df_classification_rpt.round(2)

# Afficher le DataFrame
df_classification_rpt

,precision,recall,f1-score,support
glioma_tumor,94.17,90.50,92.30,64200.00
meningioma_tumor,86.27,89.80,88.00,49000.00
no_tumor,98.74,96.32,97.52,40800.00
pituitary_tumor,94.79,97.51,96.13,52200.00
accuracy,93.26,93.26,93.26,93.26
macro avg,93.49,93.53,93.49,206200.00
weighted avg,93.35,93.26,93.28,206200.00
